In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms,models
from torch.utils.data import DataLoader
import sys

In [5]:
#数据预处理
transform=transforms.Compose(
    [transforms.RandomResizedCrop(224),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()]
)

#读取数据
root='image'
train_dataset=datasets.ImageFolder(root+'/train',transform)
test_dataset=datasets.ImageFolder(root+'/test',transform)
print(len(train_dataset),len(test_dataset))

#导入数据
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=8,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=8,shuffle=True)

2000 1000


In [6]:
train_dataset.classes


['cats', 'dogs']

In [7]:
train_dataset.class_to_idx

{'cats': 0, 'dogs': 1}

In [8]:
model=models.vgg16(pretrained=True)
model=model.cuda()
print(model)

D:\anaconda_\envs\torch1.12\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
D:\anaconda_\envs\torch1.12\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [13]:
#冻结卷积层参数
for param in model.parameters():
    param.requires_grad=False


#构建新的全连接层    
model.classifier=torch.nn.Sequential(torch.nn.Linear(25088,100),
                                     torch.nn.ReLU(),
                                     torch.nn.Dropout(p=0.5),
                                     torch.nn.Linear(100,2)
                                     )
model=model.cuda()

In [14]:
LR=0.0003
loss_en=nn.CrossEntropyLoss()
loss_en=loss_en.cuda()
optim=optim.Adam(model.parameters(),LR)

In [15]:
def train():
    model.train()#启用dropout和BN
    for i,data in enumerate(train_loader):
        inputs,labels=data
        inputs,labels=inputs.cuda(),labels.cuda()
        out=model(inputs)
        loss=loss_en(out,labels)
        loss=loss.cuda()
        optim.zero_grad()
        loss.backward()
        optim.step()

def test():
    model.eval()
    correct=0
    for i,data in enumerate(test_loader):
        inputs,labels=data
        inputs,labels=inputs.cuda(),labels.cuda()
        out=model(inputs)
        _,predicted=torch.max(out,1)
        correct+=(predicted==labels).sum()
    print('test acc: {}'.format(correct.item()/len(test_dataset)))
    
    correct=0
    for i,data in enumerate(train_loader):
        inputs,labels=data
        inputs,labels=inputs.cuda(),labels.cuda()
        out=model(inputs)
        _,predicted=torch.max(out,1)
        correct+=(predicted==labels).sum()
    print('train acc: {}'.format(correct.item()/len(train_dataset)))
    
    

In [16]:
for epoch in range(10):
    print('epoch:',epoch)
    train()
    
    test()
torch.save(model.state_dict(),'cat_dog.pth')

epoch: 0
test acc: 0.865


KeyboardInterrupt: 